<a href="https://colab.research.google.com/github/napoler/mlmxl-pytorch/blob/master/Memory_Transformer_XL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!Memory Transformer-XL

/bin/bash: Memory: command not found


In [2]:
!pip install memory-transformer-xl
!pip install transformers
!pip install mlm-pytorch

In [3]:
import torch
from memory_transformer_xl import MemoryTransformerXL

model = MemoryTransformerXL(
    num_tokens = 20000,
    dim = 1024,
    heads = 8,
    depth = 8,
    seq_len = 512,
    mem_len = 256,            # short term memory (the memory from transformer-xl)
    lmem_len = 256,           # long term memory (memory attention network attending to short term memory and hidden activations)
    mem_write_iters = 2,      # number of iterations of attention for writing to memory
    memory_layers = [6,7,8],  # which layers to use memory, only the later layers are actually needed
    num_mem_kv = 128,         # number of memory key/values, from All-attention paper

).cuda()

x1 = torch.randint(0, 20000, (1, 512)).cuda()
logits1, mem1 = model(x1)

x2 = torch.randint(0, 20000, (1, 512)).cuda()
logits2, mem2 = model(x2, memories = mem1)

In [4]:
mem2

Memory(short=tensor([[[[ 0.0362,  0.5993,  0.2789,  ..., -0.0101,  0.3508, -0.3563],
          [-0.0494,  0.2674,  0.4897,  ..., -0.4644, -0.7191, -0.4886],
          [-0.3707,  0.2959,  0.2633,  ...,  0.2003, -0.4072, -0.5428],
          ...,
          [ 0.3973,  0.6157,  0.5027,  ...,  0.5756, -0.5070, -0.6027],
          [ 0.1458, -0.1717, -0.2221,  ..., -0.1036, -0.4442, -0.0377],
          [-0.3231, -0.1347,  0.6932,  ..., -0.3780,  0.1821, -0.5134]]],


        [[[ 0.0937,  0.5486,  0.1062,  ...,  0.0058,  0.3140, -0.3302],
          [-0.0363,  0.2995,  0.4309,  ..., -0.3821, -0.5070, -0.4848],
          [-0.4930,  0.2653,  0.2242,  ...,  0.1923, -0.4976, -0.4233],
          ...,
          [ 0.3066,  0.5526,  0.5867,  ...,  0.5526, -0.5247, -0.5976],
          [ 0.2169, -0.1093, -0.2304,  ..., -0.0869, -0.4577, -0.0665],
          [-0.1896, -0.1146,  0.6905,  ..., -0.3956,  0.1697, -0.5003]]],


        [[[ 0.0673,  0.5409,  0.1710,  ...,  0.0041,  0.3060, -0.3003],
          [-0

# 分词
数据特点：
可直接用于预训练、语言模型或语言生成任务。
发布专用于简体中文NLP任务的小词表。
词表介绍
Google原始中文词表和我们发布的小词表的统计信息如下：

Token Type	Google	CLUE
Simplified Chinese	11378	5689
Traditional Chinese	3264	✗
English	3529	1320
Japanese	573	✗
Korean	84	✗
Emoji	56	✗
Numbers	1179	140
Special Tokens	106	106
Other Tokens	959	766
Total	21128	8021
https://github.com/CLUEbenchmark/CLUEPretrainedModels

In [5]:
from transformers import AutoTokenizer, AutoModel,BertTokenizer
  
tokenizer = BertTokenizer.from_pretrained("clue/roberta_chinese_clue_tiny")

# model = AutoModel.from_pretrained("clue/roberta_chinese_clue_tiny")

In [6]:
tokenizer

PreTrainedTokenizer(name_or_path='clue/roberta_chinese_clue_tiny', vocab_size=8021, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [7]:
tokenizer.vocab_size

8021

In [8]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_batch_encode_plus',
 '_batch_prepare_for_model',
 '_bos_token',
 '_cls_token',
 '_convert_id_to_token',
 '_convert_token_to_id',
 '_convert_token_to_id_with_added_voc',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_from_pretrained',
 '_get_padding_truncation_strategies',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad_token_type_id',
 '_push_to_hub',
 '_save_pretrained',
 '_sep_token',
 '_tokenize',
 '_unk_token',
 'add_speci

# 模型测试

使用mlm模式训练
https://github.com/lucidrains/mlm-pytorch

In [10]:
# vocab_size

## 重新mlm模型



In [14]:
import math
from functools import reduce

import torch
from torch import nn
import torch.nn.functional as F

# helpers

def prob_mask_like(t, prob):
    return torch.zeros_like(t).float().uniform_(0, 1) < prob

def mask_with_tokens(t, token_ids):
    init_no_mask = torch.full_like(t, False, dtype=torch.bool)
    mask = reduce(lambda acc, el: acc | (t == el), token_ids, init_no_mask)
    return mask

def get_mask_subset_with_prob(mask, prob):
    batch, seq_len, device = *mask.shape, mask.device
    max_masked = math.ceil(prob * seq_len)

    num_tokens = mask.sum(dim=-1, keepdim=True)
    mask_excess = (mask.cumsum(dim=-1) > (num_tokens * prob).ceil())
    mask_excess = mask_excess[:, :max_masked]

    rand = torch.rand((batch, seq_len), device=device).masked_fill(~mask, -1e9)
    _, sampled_indices = rand.topk(max_masked, dim=-1)
    sampled_indices = (sampled_indices + 1).masked_fill_(mask_excess, 0)

    new_mask = torch.zeros((batch, seq_len + 1), device=device)
    new_mask.scatter_(-1, sampled_indices, 1)
    return new_mask[:, 1:].bool()

# main class

class MLMXL(nn.Module):
    def __init__(
        self,
        transformer,
        mask_prob = 0.15,
        replace_prob = 0.9,
        num_tokens = None,
        random_token_prob = 0.,
        mask_token_id = 2,
        pad_token_id = 0,
        mask_ignore_token_ids = []):
        super().__init__()

        self.transformer = transformer
        self.mem=None

        # mlm related probabilities
        self.mask_prob = mask_prob
        self.replace_prob = replace_prob

        self.num_tokens = num_tokens
        self.random_token_prob = random_token_prob

        # token ids
        self.pad_token_id = pad_token_id
        self.mask_token_id = mask_token_id
        self.mask_ignore_token_ids = set([*mask_ignore_token_ids, pad_token_id])

    def forward(self, input, **kwargs):
        # do not mask [pad] tokens, or any other tokens in the tokens designated to be excluded ([cls], [sep])
        # also do not include these special tokens in the tokens chosen at random
        no_mask = mask_with_tokens(input, self.mask_ignore_token_ids)
        mask = get_mask_subset_with_prob(~no_mask, self.mask_prob)

        # get mask indices
        mask_indices = torch.nonzero(mask, as_tuple=True)

        # mask input with mask tokens with probability of `replace_prob` (keep tokens the same with probability 1 - replace_prob)
        masked_input = input.clone().detach()

        # if random token probability > 0 for mlm
        if self.random_token_prob > 0:
            assert self.num_tokens is not None, 'num_tokens keyword must be supplied when instantiating MLM if using random token replacement'
            random_token_prob = prob_mask_like(input, self.random_token_prob)
            random_tokens = torch.randint(0, self.num_tokens, input.shape, device=input.device)
            random_no_mask = mask_with_tokens(random_tokens, self.mask_ignore_token_ids)
            random_token_prob &= ~random_no_mask
            random_indices = torch.nonzero(random_token_prob, as_tuple=True)
            masked_input[random_indices] = random_tokens[random_indices]

        # [mask] input
        replace_prob = prob_mask_like(input, self.replace_prob)
        masked_input = masked_input.masked_fill(mask * replace_prob, self.mask_token_id)

        # mask out any tokens to padding tokens that were not originally going to be masked
        labels = input.masked_fill(~mask, self.pad_token_id)
        if self.mem!=None:
        # get generator output and get mlm loss
          logits,self.mem = self.transformer(masked_input, memories = self.mem, **kwargs)
        else:
          logits,self.mem = self.transformer(masked_input, **kwargs)

        mlm_loss = F.cross_entropy(
            logits.transpose(1, 2),
            labels,
            ignore_index = self.pad_token_id
        )

        return mlm_loss


In [11]:
import torch
from memory_transformer_xl import MemoryTransformerXL
import torch
from torch import nn
from torch.optim import Adam
# from mlm_pytorch import MLM

model = MemoryTransformerXL(
    num_tokens = tokenizer.vocab_size,
    dim = 128,
    heads = 8,
    depth = 8,
    seq_len = 1024,
    mem_len = 256,            # short term memory (the memory from transformer-xl)
    lmem_len = 256,           # long term memory (memory attention network attending to short term memory and hidden activations)
    mem_write_iters = 2,      # number of iterations of attention for writing to memory
    memory_layers = [6,7,8],  # which layers to use memory, only the later layers are actually needed
    num_mem_kv = 128,         # number of memory key/values, from All-attention paper

).cuda()

x1 = torch.randint(0, tokenizer.vocab_size, (1, 1024)).cuda()
logits1, mem1 = model(x1)

x2 = torch.randint(0, tokenizer.vocab_size, (1, 1024)).cuda()
logits2, mem2 = model(x2, memories = mem1)

In [ ]:
tokenizer

In [ ]:
torch.save(model.state_dict(), "model1024.bin")

In [16]:
# plugin the language model into the MLM trainer

trainer = MLMXL(
    model,
    mask_token_id = tokenizer.mask_token_id,          # the token id reserved for masking
    pad_token_id = tokenizer.pad_token_id,           # the token id for padding
    mask_prob = 0.15,           # masking probability for masked language modeling
    replace_prob = 0.90,        # ~10% probability that token will not be masked, but included in loss, as detailed in the epaper
    mask_ignore_token_ids = [tokenizer.cls_token_id,tokenizer.sep_token_id]  # other tokens to exclude from masking, include the [cls] and [sep] here
).cuda()

# optimizer

opt = Adam(trainer.parameters(), lr=3e-4)

# one training step (do this for many steps in a for loop, getting new `data` each time)

data = torch.randint(0, tokenizer.vocab_size, (2, 1024)).cuda()

loss = trainer(data)
loss.backward()
opt.step()
opt.zero_grad()

# after much training, the model should have improved for downstream tasks

# torch.save(transformer, f'./pretrained-model.pt')

In [17]:
loss

tensor(9.0243, device='cuda:0', grad_fn=<NllLoss2DBackward>)

In [ ]:
# dir(model)
model

In [ ]:
logits2